## Pytorch활용한 MNIST 데이터셋 로딩

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms

root = './data'
mnist_train = dset.MNIST (root=root, train=True, transform=transforms.ToTensor(), download=True )
mnist_test = dset.MNIST (root=root, train=False, transform=transforms.ToTensor(), download=True)

# Train용 / Test용 데이터셋
train_loader = DataLoader(mnist_train, batch_size=10, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=10, shuffle=True)

## 학습준비(가중치 초기화 등)
 * MNIST의 크기 : 28 * 28
 * Loss : Cross Entropy
 * Optimizer - SGD(Stochastic Gradient Descent)
 * Learning rate = 0.1

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 구현 (28*28 = 784 / 0~9라서 10개 / 가중치 사용하므로 bias)
linear = torch.nn.Linear(784, 10, bias=True).to(device) 

# weight init 가중치 초기화
torch.nn.init.normal_(linear.weight)

# Loss fn - Cross Entropy Loss
criterion = torch.nn.CrossEntropyLoss().to(device)

# optimizer - SGD
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

## 모델 학습

In [7]:
training_epochs = 20 # training 반복 횟수

for epoch in range(training_epochs):
  for i, (imgs, labels) in enumerate(train_loader):
    labels = labels.to(device)
    imgs = imgs.view(-1, 28 * 28).to(device)

    outputs = linear(imgs) 
    loss = criterion(outputs, labels) 

    optimizer.zero_grad()# optimzier zero grad

    loss.backward() # loss backward
    optimizer.step() # optimzier step

    _,argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax).float().mean()

  if (i+1) % 100 == 0:
    print('Epoch [{}/{}], Step [{}/{}], Loss: {: .4f}, Accuracy: {: .2f}%'.format(
    epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item()* 100))

Epoch [1/20], Step [6000/6000], Loss:  0.0273, Accuracy:  100.00%
Epoch [2/20], Step [6000/6000], Loss:  0.0762, Accuracy:  100.00%
Epoch [3/20], Step [6000/6000], Loss:  0.5928, Accuracy:  80.00%
Epoch [4/20], Step [6000/6000], Loss:  0.2854, Accuracy:  90.00%
Epoch [5/20], Step [6000/6000], Loss:  0.1373, Accuracy:  90.00%
Epoch [6/20], Step [6000/6000], Loss:  0.0668, Accuracy:  100.00%
Epoch [7/20], Step [6000/6000], Loss:  0.0253, Accuracy:  100.00%
Epoch [8/20], Step [6000/6000], Loss:  0.0542, Accuracy:  100.00%
Epoch [9/20], Step [6000/6000], Loss:  0.9203, Accuracy:  80.00%
Epoch [10/20], Step [6000/6000], Loss:  0.1244, Accuracy:  90.00%
Epoch [11/20], Step [6000/6000], Loss:  0.6108, Accuracy:  90.00%
Epoch [12/20], Step [6000/6000], Loss:  0.1312, Accuracy:  100.00%
Epoch [13/20], Step [6000/6000], Loss:  0.0705, Accuracy:  100.00%
Epoch [14/20], Step [6000/6000], Loss:  1.6259, Accuracy:  70.00%
Epoch [15/20], Step [6000/6000], Loss:  0.0538, Accuracy:  100.00%
Epoch [16/2

## 학습된 모델 테스트

In [8]:
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs) # 구현

        _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0)
        correct += (labels == argmax). sum().item()

    print('Test accuracy for {} images: {: .2f}%'.format(total, correct / total * 100))

Test accuracy for 10000 images:  91.99%
